In [2]:
import import_ipynb
import pickle
import pandas as pd
from default_prediction import predict_default
from datetime import datetime
from tabulate import tabulate
from db_setup import get_collections
from interest_rate_model_input import predict_interest_rate
collections = get_collections()
loan_requests_collection = collections["loan_requests"]
payments_collection = collections["payments"]
borrower_profiles_collection = collections["borrower_profiles"]

# Method to create a loan request
def create_loan_request(user):
    borrower_id = user["user_id"]
    requested_amount = float(input("Enter Requested Loan Amount: "))
    loan_term = int(input("Enter Loan Term (Ex: 12, 24, 36, 48, 60): "))
    homeownership = input("Enter Homeownership Status (e.g., RENT, OWN): ").lower()
    verified_income = input("Is income verified (True/False): ").lower()
    purpose = input("Enter Purpose of Loan (e.g., Auto, Business, Home, Education, Other): ").lower()
    grade = input("Enter grade associated with loan: ").lower()
    sub_grade = input("Enter detailed grade associated with loan: ").lower()
    has_cosigner = input("Do you have Co-signer (Yes/ No): ").lower()
    has_dependants = input("Do you have Dependants (Yes/ No): ").lower()

    loan_request_id = f"LR_{borrower_id}_{loan_requests_collection.count_documents({}) + 1:04}"

    borrower_profile = borrower_profiles_collection.find_one({"borrower_id": borrower_id})
    if not borrower_profile:
        print(f"No borrower profile found for Borrower ID: {borrower_id}")
        return

    # If the borrower is new to platform, their initial debt-to-income-ratio and total number of credit lines in the platform will be 0
    debt_to_income = borrower_profile.get('debt_to_income_ratio', 0.0)
    num_credit_lines = borrower_profile.get('num_credit_lines', 0.0)

    # Input to interest rate prediction model
    input_values = {
        "grade": grade,
        "sub_grade": sub_grade,
    }
    # Predict the interest rate
    interest_rate =  round(float(predict_interest_rate(input_values)), 2)

    # Input to default on prediction
    default_user_input = {
            'age': user['age'],
            'income': user['annual_income'],
            'loan_amount': requested_amount,
            'credit_score': user['credit_score'],
            'emp_length': user['emp_length'],
            'num_credit_lines': num_credit_lines,
            'interest_rate': interest_rate,
            'loan_term': loan_term,
            'debt_to_income_ratio': debt_to_income,
            'education': user['education'],
            'emp_type': user['emp_type'],
            'has_dependents': has_dependants,
            'purpose': purpose,
            'has_co_signer': has_cosigner
        }

    # Predict the default on value
    default = predict_default(default_user_input)

    if not default:
        # The loan request made by user is predicted to be defaulted
        return
        
    print("\nPredicted Interest Rate:", round(interest_rate, 2), "%")
    loan_request = {
        "loan_request_id": loan_request_id,
        "borrower_id": borrower_id,
        "requested_amount": requested_amount,
        "predicted_interest_rate": interest_rate,
        "loan_term": loan_term,
        "homeownership": homeownership,
        "verified_income": verified_income,
        "purpose": purpose,
        "grade": grade,
        "sub_grade": sub_grade,
        "has_co_signer": has_cosigner,
        "has_dependants": has_dependants,
        "status": "pending",
        "requested_date": datetime.now().strftime("%Y-%m-%d")
    }

    # Insert the loan request to the loan requests collection
    loan_requests_collection.insert_one(loan_request)
    print(f"Loan Request {loan_request_id} successfully created for Borrower ID: {borrower_id}")

# Method to track status of all loan requests made by the borrower
def track_loan_status(user):
    requests = loan_requests_collection.find({"borrower_id": user["user_id"]})
    rows = []
    for req in requests:
        rows.append([req["loan_request_id"], req["borrower_id"], req["requested_amount"], req.get("predicted_interest_rate", 0), req["loan_term"], req["purpose"], req["status"]])
    if rows:
        print(tabulate(rows, headers=["Request ID", "Borrower ID", "Amount", "Predicted Interest Rate", "Term", "Purpose", "Status"], tablefmt="grid"))
    else:
        print("No loan requests available.")

# Method to calculate the number of remaining installments for the complete loan to be repaid/
def calculate_repayment_schedule(user):
    payments = payments_collection[{"borrower_id": user["user_id"]}]
    repayment = []
    payment_record = payments_collection.find_one({"borrower_id": user["user_id"]})
    
    if not payment_record:
        print(f"No payment record found for borrower_id: {user["user_id"]}")
        return

    payment_profile = payment_record.get("payment_profile", [])
    if not payment_profile:
        print(f"No loans found for borrower_id: {user["user_id"]}")
        return
    
    table_data = []
    for loan in payment_profile:
        amount_remaining = float(loan.get("balance", 0))
        remaining_installments = amount_remaining /  loan["installment"]
       
        table_data.append([
            loan.get("loan_id", "N/A"),
            loan.get("loan_amount", "N/A"),
            loan.get("paid_total", "N/A"),
            loan.get("paid_interest", "N/A"),
            loan.get("paid_principal", "N/A"),
            loan.get("installment", "N/A"),
            f"{amount_remaining:.2f}",
            f"{remaining_installments:.0f}" 
        ])
    headers = ["Loan ID", "Loan Amount", "Paid Total", "Paid Interest", "Paid Principal", "Installment", "Balance Loan Amount", "Remaining Installments"]
    
    print(f"Loans for Borrower ID: {user["user_id"]}")
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

# Method to delete the loan request
def delete_loan_request(user):
    borrower_loan_requests(user)
    loan_requests = list(loan_requests_collection.find({"status": "pending", "borrower_id": user["user_id"]}))
    if not loan_requests:
        return
    loan_request_ids = input("Enter Loan Request ID/IDs to delete (comma-separated): ").strip().split(",")

    loan_request_ids = [loan_id.strip() for loan_id in loan_request_ids if loan_id.strip()]
    if not loan_request_ids:
        print("Error: No valid loan request IDs provided.")
        return

    query = {"loan_request_id": {"$in": loan_request_ids}}

    result = loan_requests_collection.delete_many(query)

    if result.deleted_count > 0:
        print(f"Successfully deleted {result.deleted_count} loan request(s).")
    else:
        print("No loan requests found for the given IDs.")


# Method to display all the loan requests made by the borrower
def borrower_loan_requests(user):
    loan_requests = list(loan_requests_collection.find({"status": "pending", "borrower_id": user["user_id"]}))
    

    if not loan_requests:
        print("No loan requests available.")
        return
    
    table_data = []
    for request in loan_requests:
        table_data.append([
            request.get("loan_request_id", "N/A"),
            request.get("borrower_id", "N/A"),
            request.get("requested_amount", "N/A"),
            request.get("predicted_interest_rate", "N/A"),
            request.get("loan_term", "N/A"),
            request.get("homeownership", "N/A"),
            request.get("verified_income", "N/A"),
            request.get("purpose", "N/A"),
            request.get("grade", "N/A"),
            request.get("sub_grade", "N/A"),
            request.get("status", "N/A"),
            request.get("requested_date", "N/A"),
        ])
    
    headers = [
        "Request ID", "Borrower ID", "Requested Amount", "Predicted Interest Rate",
        "Loan Term", "Homeownership", "Verified Income",
        "Purpose", "Grade", "Sub Grade", "Status", "Requested Date"
    ]
    
    print(tabulate(table_data, headers=headers, tablefmt="grid"))